In [1]:
import os
%pwd

'/Users/kishorreddy/Desktop/Chicken-Disease-Classification/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/kishorreddy/Desktop/Chicken-Disease-Classification'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen= True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: Path
    params_learning_rate: Path
    params_include_top: bool
    params_weights: str
    arams_classes: int 
    

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [18]:
class ConfiguratioManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath),
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
        )
          
        return prepare_base_model_config




In [20]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [ ]:
class PreapreBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        
        self.save_mode(path = self.config.base_model_path, model = self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, earning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax"
            )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            output = prediction
        )

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(Learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )
        self.save_model(path = self.config.updated_base_model_path, model = self.full_model)
        
        
        

